In [ ]:
#Brand Pulse Dashboard
#Import Necessary Packages
import pandas as pd
import numpy as np
import statistics as st
import calendar
import datetime
import re
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [ ]:
## Loading Data
PastYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/PastYearDataNew.csv", encoding='utf-8')	
CurrentYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/CurrentYearDataNew.csv", encoding='utf-8')	

#Combine Past and recent years sales data
SD = pd.concat([PastYearData, CurrentYearData], ignore_index=True)
SDnum = pd.concat([PastYearData, CurrentYearData], ignore_index=True)

SkuAddDate =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/skuAddDate.csv", encoding='utf-8')	


In [ ]:
# Convert Order_Date string to date and extract relevant date values
#https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
SD['Order_Date'] = pd.to_datetime(SD['Order_Date'], format= "%d-%b-%y")
SD['OD_Year'] = SD['Order_Date'].dt.strftime('%Y')
SD['OD_MonthNum'] = SD['Order_Date'].dt.strftime('%m')
SD['OD_MonthLab'] = SD['Order_Date'].dt.strftime('%B')
SD['OD_MonthDay'] = SD['Order_Date'].dt.strftime('%d')
SD['OD_WeekDay'] = SD['Order_Date'].dt.strftime('%A')
SD["NetRevenue"]= SD.Total_Net_Price_CAD_ - SD.Total_Refunded_CAD_


In [ ]:
# Filter out any blank orderdate values
FilterSD =    SD[(SD['Order_Date']!="")
           #& (SD['Supplier']=="")
           ]

In [ ]:
#Extract meta data from SD
##Identify all unique brands
AllBrands = SD.attribute_set.unique().astype(str)
AllBrands = AllBrands[(AllBrands!="Discontinued") & (AllBrands!="nan") ]

AllBrands[268]

AllSuppliers = SD.Supplier.unique()
AllBucketValues = SD.Order_Bucket.unique()

#list(SD.columns) 

In [ ]:
#Group by Attribute set & year sold, then count number of sales
monthlyGroupingN = (FilterSD.groupby(["attribute_set", "OD_MonthNum", "OD_MonthDay"], as_index=False)
                ['Order_Date'].
                agg({"count":"count"}).sort_values(["attribute_set","OD_MonthNum","OD_MonthDay"] , ascending = True))

monthlyGroupingR = (FilterSD.groupby(["attribute_set", "OD_MonthNum", "OD_MonthDay"], as_index=False)
                ['NetRevenue'].
                agg({"NetRevenue":"sum"}).sort_values(["attribute_set","OD_MonthNum", "OD_MonthDay"] , ascending = True))

monthlyGroupingAll = monthlyGroupingN.merge(monthlyGroupingR, on=('attribute_set', 'OD_MonthNum', "OD_MonthDay")).reset_index(drop=True)


In [ ]:

DueDatePrediction = pd.DataFrame(columns = ['attribute_set',"MNum",'Month' , 'Day', 
                                                "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", 
                                                "Sep", "Oct", "Nov", "Dec","TotalOAD","peakDays", "test1"])
      
for value in range(len(AllBrands)-1):

    brand = AllBrands[value] 
    BrandYearSS = monthlyGroupingAll[(monthlyGroupingN['attribute_set']==brand)].reset_index(drop=True)

    BrandYearSS["OverSales"] =  BrandYearSS["count"] - st.mean(BrandYearSS["count"])
    BrandYearSS["OverRev"] = BrandYearSS["NetRevenue"] -  st.mean(BrandYearSS["NetRevenue"]) 

    OverIdentify = BrandYearSS[(BrandYearSS["OverSales"]>0) & (BrandYearSS["OverRev"]>0)]
    OverIdentify_Tally = OverIdentify.groupby(["OD_MonthNum"], as_index=False)['OD_MonthNum'].agg({"OverCounter":"count"})
    
    OverIdentify_Final = pd.DataFrame({"OD_MonthNum":["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]})
    OverIdentify_Final = OverIdentify_Final.merge(OverIdentify_Tally, on=('OD_MonthNum'), how = "outer").reset_index(drop=True)
    OverIdentify_Final = OverIdentify_Final.fillna(0)
             
    if len(OverIdentify_Final)>1:
    
        for m in range(len(OverIdentify_Final)-1):
            Base = (OverIdentify_Final.loc[m+1, "OverCounter"])
            Threshold = (OverIdentify_Final.loc[m, "OverCounter"]*1.5)
            if Base>Threshold:
                if Base>=6:###Consider reducing to 6
                    OverIdentify_Final.loc[m, "test1"] = "Update"
                else:
                    OverIdentify_Final.loc[m, "test1"]  = ""
            else:
                OverIdentify_Final.loc[m, "test1"]  = ""

        FindDup = len(OverIdentify_Final[OverIdentify_Final["test1"]=="Update"])
        
        if FindDup > 0:
            if FindDup==1:
                OverIdentify_Final["test2"] = OverIdentify_Final["test1"]

                WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)

            elif FindDup>1:
                MultiUpdate = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"]
                MaxMonths = (len(OverIdentify_Final)-1 - MultiUpdate.index[-1])+1
                MultiUpdate["index"] = MultiUpdate.index
                MultiUpdate = MultiUpdate.reset_index(drop=True)

                for s in range(len(MultiUpdate)):
                    if s != (len(MultiUpdate)-1):
                        IndexVal = MultiUpdate.loc[s,"index"]
                        MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal), "OverCounter"])
                    else: 
                        IndexVal = MultiUpdate.loc[s,"index"]
                        MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal-1), "OverCounter"], OverIdentify_Final.loc[0, "OverCounter"])                  

                WinMonth = MultiUpdate.sort_values("Value", ascending = False).reset_index(drop=True).loc[0,"index"]
                
                for m in range(len(OverIdentify_Final)-1):
                    if m == WinMonth:
                        OverIdentify_Final.loc[WinMonth, "test2"] = "Update"
                    else:
                        OverIdentify_Final.loc[m, "test2"]  = ""

                WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)


            #Determine the end of peak
            count = 0
            sumDays = 0
            dayCount = 0
            
            for i in range(int(WinningMonth), len(OverIdentify_Final)):
                if i != 12:
                    month = OverIdentify_Final.loc[i, "OD_MonthNum"]
                    days = OverIdentify_Final.loc[i, "OverCounter"]
                    count = count + 1
                    sumDays = sumDays + days
                    avg = (sumDays/count)*.75

                    if(days >= avg): 
                        dayCount = dayCount + 30
                    else:
                        break
                elif i == 12:
                    month = OverIdentify_Final.loc[0, "OD_MonthNum"]
                    days = OverIdentify_Final.loc[0, "OverCounter"]
                    count = count + 1
                    sumDays = sumDays + days
                    avg = (sumDays/count)*.75

                    if(days >= avg):
                        dayCount = dayCount + 30
                    else:
                        break
                
                
            #Determine day of WinningMonth of due date
            TextMonth = calendar.month_name[int(WinningMonth)]
            MaxCountIndex = SubsetWinMonth[SubsetWinMonth["count"] == (SubsetWinMonth["count"].max())].index[0]
            MaxRevIndex = SubsetWinMonth[SubsetWinMonth["NetRevenue"] == (SubsetWinMonth["NetRevenue"].max())].index[0]

            if MaxCountIndex == MaxRevIndex:
                MaxOfMonth = MaxCountIndex
                if MaxOfMonth == len(SubsetWinMonth)-1:
                    MinusLast = SubsetWinMonth.loc[0:len(SubsetWinMonth)-2]
                    MaxCountIndex = MinusLast[MinusLast["count"] == (MinusLast["count"].max())].index[0]
                    MaxRevIndex = MinusLast[MinusLast["NetRevenue"] == (MinusLast["NetRevenue"].max())].index[0]
                    if MaxCountIndex == MaxRevIndex:
                        MaxOfMonth = MaxCountIndex
                        MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                    else: 
                        MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                        MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                else:
                    MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
            else: 
                MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                MinSubset = SubsetWinMonth.loc[MaxOfMonth:]

            MinCountIndex = MinSubset[MinSubset["count"]==MinSubset.loc[MaxOfMonth:, "count"].min()].index[-1]
            MinRevIndex =   MinSubset[MinSubset["NetRevenue"]==MinSubset.loc[MaxOfMonth:, "NetRevenue"].min()].index[-1]    

            if MinCountIndex == MinRevIndex:
                FinalIndex = MinRevIndex
                SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
            else: 
                FinalIndex = min(MinCountIndex, MinRevIndex)
                SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
        else:
            TextMonth = "None"
            WinningMonth = 0
            WinDay = 0
        
    #Formatting Data
    AllRecomMon = str()
    AllRecomMonRou = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"].reset_index(drop = True)
    for m in range(len(AllRecomMonRou)):
        AllRecomMon = AllRecomMon + calendar.month_name[int(AllRecomMonRou.loc[m,"OD_MonthNum"])]+ "-"
            
    OC = OverIdentify_Final["OverCounter"]
    TotalOAD = sum(OC)
    DueDateData = brand,WinningMonth,TextMonth,WinDay,OC[0],OC[1],OC[2],OC[3],OC[4],OC[5],OC[6],OC[7],OC[8],OC[9],OC[10],OC[11],TotalOAD,dayCount, str(AllRecomMon)
    
    DueDatePrediction = DueDatePrediction.append(pd.Series(DueDateData, index=DueDatePrediction.columns),  ignore_index=True)
    Daydata = DueDatePrediction[["attribute_set","peakDays"]]
                        

In [ ]:
#subset internal sku, attribute set and order date from SD data frame
SoldSkus = pd.DataFrame(SD.internal_sku.unique().astype(str))
SoldSkus.columns = ["internal_sku"]

#Merge SoldSkus with SkuAddDate
SoldSkus = pd.merge(SoldSkus, SkuAddDate, how='inner', on="internal_sku")
SoldSkus['add_date'] = pd.to_datetime(SoldSkus['add_date'], format= "%Y-%m-%d")
BrandLabel = SD[["internal_sku", "attribute_set"]]
SoldSkus = pd.merge(SoldSkus, BrandLabel, how='inner', on="internal_sku").drop_duplicates(keep=False)
SoldSkus= SoldSkus.reset_index()

for i in range(len(SoldSkus)):
    sku = SoldSkus.loc[i,"internal_sku"]
    Brand = SoldSkus.loc[i,"attribute_set"]
    AddDate = SoldSkus.loc[i,"add_date"]

    AllOrderDates = SD[SD["internal_sku"]==sku].Order_Date.sort_values().reset_index().loc[0,"Order_Date"]
    PeakDayValue = Daydata[Daydata["attribute_set"]==Brand].reset_index().loc[0, "peakDays"]
    
    RevSubset = SD[SD["internal_sku"]==sku][["Order_Date", "NetRevenue" ]]
    RevSubset["SellDistance"] = abs(RevSubset["Order_Date"]-AddDate).dt.days
    RevSubset["SellDifference"] = PeakDayValue -  RevSubset["SellDistance"]     
    NewSkuRev = sum(RevSubset[RevSubset["SellDifference"] >= 0]["NetRevenue"])
    
    SoldSkus.loc[i,"FirstSoldDate"] = AllOrderDates
    SoldSkus.loc[i,"PeakDayRange"] = PeakDayValue
    SoldSkus.loc[i,"NewSkuRev"] = NewSkuRev
    
    print(SoldSkus.loc[i,"internal_sku"], "         ", NewSkuRev)


SoldSkus['DayDiff'] = abs(SoldSkus.FirstSoldDate-SoldSkus.add_date).dt.days


for i in range(len(SoldSkus)):
    DayDiff = SoldSkus.loc[i,"DayDiff"]
    pdr = SoldSkus.loc[i,"PeakDayRange"]
    
    if DayDiff <= pdr: 
        SoldSkus.loc[i, "UnderPeakDays"] = 1
    else: 
        SoldSkus.loc[i, "UnderPeakDays"] = 0
        
    if (DayDiff<365):
        SoldSkus.loc[i, "UnderYear"] = 1
        SoldSkus.loc[i, "OverYear"] = 0
        
        if(DayDiff<180):
            SoldSkus.loc[i, "HalfYear"] = 1
        else:
            SoldSkus.loc[i, "HalfYear"] = 0
    else:
        SoldSkus.loc[i, "UnderYear"] = 0 
        SoldSkus.loc[i, "OverYear"] = 1        


AvgDays = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['DayDiff'].
                agg({"AvgDayDiff":"mean"}).sort_values(["AvgDayDiff"] , ascending = True))
    
TimesSold = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['DayDiff'].
                agg({"TimesSold":"count"}).sort_values(["TimesSold"] , ascending = True))

UnderYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['UnderYear'].
                agg({"UnderYear":"sum"}))
    
OverYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['OverYear'].
                agg({"OverYear":"sum"}))

HalfYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['HalfYear'].
                agg({"HalfYear":"sum"}))

UnderPeakDays = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['UnderPeakDays'].
                agg({"UnderPeakDays":"sum"}))

NewSkuRev = (SoldSkus.groupby(["attribute_set"], as_index=False)
                ['NewSkuRev'].
                agg({"NewSkuRev":"sum"}))

finalskuCount = AvgDays.merge(TimesSold, on="attribute_set" )
finalskuCount = finalskuCount.merge(UnderYear, on="attribute_set" )
finalskuCount = finalskuCount.merge(OverYear, on="attribute_set" )
finalskuCount = finalskuCount.merge(HalfYear, on="attribute_set" )
finalskuCount = finalskuCount.merge(UnderPeakDays, on="attribute_set" )
finalskuCount = finalskuCount.merge(NewSkuRev, on="attribute_set" )
finalskuCount["%UnderYear"] = finalskuCount["UnderYear"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
finalskuCount["%HalfYear"] = finalskuCount["HalfYear"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
finalskuCount["%UnderPeakDays"] = finalskuCount["UnderPeakDays"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
finalskuCount = finalskuCount.sort_values(["TimesSold", "AvgDayDiff", "UnderYear"] , ascending = [False,True, True])


In [ ]:
#Aquire net revenue for all brands
BrandRev = monthlyGroupingAll.groupby("attribute_set")["NetRevenue"].agg("sum")
FinalBrandProfile = finalskuCount.merge(BrandRev, on="attribute_set" ).merge(DueDatePrediction, on="attribute_set" )
FinalBrandProfile["%NewSkuRev"] = (FinalBrandProfile["NewSkuRev"]/FinalBrandProfile["NetRevenue"])*100
FinalBrandProfile["%NewSkuRev"] = (FinalBrandProfile["NewSkuRev"]/FinalBrandProfile["NetRevenue"])*100

UpdatePrioritydf = FinalBrandProfile[["attribute_set", "MNum", "Day", "NewSkuRev", "%NewSkuRev"]]
UpdatePrioritydf["MNum"] = pd.to_numeric(UpdatePrioritydf["MNum"])
UpdatePrioritydf["Day"] = pd.to_numeric(UpdatePrioritydf["Day"])
UpdatePrioritydf = UpdatePrioritydf[(UpdatePrioritydf["NewSkuRev"]>0) & (UpdatePrioritydf["MNum"] != 0)]

UpdatePrioritydf = UpdatePrioritydf.sort_values(["MNum", "Day", "NewSkuRev"], ascending = [True, True, False]).reset_index()
#UpdatePrioritydf.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\UpdatePrioritydf.csv', index = None, header=True)

UpdatePrioritydf.loc[(UpdatePrioritydf["MNum"]==2) & (UpdatePrioritydf["Day"]==29), "Day"] = 28
UpdatePrioritydf["Year"] = year
UpdatePrioritydf["MNum"] = UpdatePrioritydf["MNum"].map("{:02}".format)
UpdatePrioritydf["Day"] = UpdatePrioritydf["Day"].map("{:02}".format)

UpdatePrioritydf["Year"] = UpdatePrioritydf["Year"].astype(str)
UpdatePrioritydf["MNum"] = UpdatePrioritydf["MNum"].astype(str)
UpdatePrioritydf["Day"] = UpdatePrioritydf["Day"].astype(str)

UpdatePrioritydf["Date"] = UpdatePrioritydf["Year"]+"-"+UpdatePrioritydf["MNum"]+"-"+UpdatePrioritydf["Day"] 
UpdatePrioritydf["Date"] = pd.to_datetime(UpdatePrioritydf["Date"], format= "%Y-%m-%d")
UpdatePrioritydf["Weekday"] = UpdatePrioritydf["Date"].dt.dayofweek
UpdatePrioritydf["WeekNum"] = UpdatePrioritydf["Date"].dt.week
UpdatePrioritydf["Ref"] = 0

In [ ]:
year = 2019
updateLimit = 4

for w in reversed(range(53)):
    weekBrands = UpdatePrioritydf[UpdatePrioritydf["WeekNum"]==w]
    BrandPerWeek = len(weekBrands)
    #print(BrandPerWeek)
    if(BrandPerWeek > updateLimit):
        BrandDiff = BrandPerWeek - updateLimit
        MovingBrandList = (list(weekBrands.sort_values("NewSkuRev", ascending = True)
                                .reset_index(drop=True)[0:BrandDiff]["attribute_set"]))
        UpdatePrioritydf.loc[UpdatePrioritydf["attribute_set"].isin(MovingBrandList), "WeekNum"] = w-1
    weekBrands = UpdatePrioritydf[UpdatePrioritydf["WeekNum"]==w].sort_values("NewSkuRev", ascending = False).reset_index()
    for f in range(len(weekBrands)):
        brand = weekBrands["attribute_set"][f]
        UpdatePrioritydf.loc[UpdatePrioritydf["attribute_set"]==brand, "Ref"] = f
        
UpdatePrioritydf["WeekNum"] = UpdatePrioritydf["WeekNum"].map("{:02}".format)

UpdateSchedule = pd.DataFrame(data={'Day': 1, 'WeekNum': range(0,53), 'Year': year})
UpdateSchedule["Day"] = UpdateSchedule["Day"].astype(str)
UpdateSchedule["WeekNum"] = UpdateSchedule["WeekNum"].map("{:02}".format).astype(str)
UpdateSchedule["Year"] = UpdateSchedule["Year"].astype(str)

UpdateSchedule["Date_temp"] = UpdateSchedule["Day"]+"-"+UpdateSchedule["WeekNum"]+"-"+UpdateSchedule["Year"] 
UpdateSchedule["Date_temp"] = pd.to_datetime(UpdateSchedule["Date_temp"], format= "%w-%W-%Y")
UpdateSchedule["Month"] = UpdateSchedule["Date_temp"].dt.month_name()
UpdateSchedule["Date"] = UpdateSchedule["Date_temp"].dt.day

RankedBrands = UpdatePrioritydf.pivot(index='WeekNum', columns='Ref', values='attribute_set')

UpdateScheduleFinal = (UpdateSchedule
                       .merge(RankedBrands, on=('WeekNum'))
                       .drop(columns=["Day","WeekNum","Year","Date_temp"])
                       .fillna(""))

#UpdateScheduleFinal.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\UpdateScheduleFinal.csv', index = None, header=True)




# Development

## GUI


What are the requirements?
1. Parameters
    1. Year (int)
    2. weekly limit (int)
    3. Evaluation Criteria
        1. Percent difference (Radio Buttons)
        2. Future Day Value (int)
2. Change output path
3. Progress bar for the for loop
4. Excecute button


In [ ]:
#combining GUI and algorithm with deafult setting set

#Import Necessary Packages
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading


#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"
            
            
def UpdatePredictor(): 
    import pandas as pd
    import numpy as np
    import statistics as st
    import calendar
    import datetime
    import re
    import matplotlib.pyplot as plt
    pd.options.mode.chained_assignment = None

    print("FunctionStarted")
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    #Reset.config(state="normal")
    RunPrediction.config(state="disabled")
    PathSelect.config(state="disabled")
    get_year.config(state="disabled")
    get_wLimit.config(state="disabled")
    get_dThresh.config(state="disabled")
    R1.config(state="disabled")
    R2.config(state="disabled")
    R3.config(state="disabled")
    R4.config(state="disabled")

    ## Loading Data
    print("loading Data")
    PastYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/PastYearDataNew.csv", encoding='utf-8')	
    CurrentYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/CurrentYearDataNew.csv", encoding='utf-8')	
    #Combine Past and recent years sales data
    SD = pd.concat([PastYearData, CurrentYearData], ignore_index=True)
    SDnum = pd.concat([PastYearData, CurrentYearData], ignore_index=True)
    SkuAddDate =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/skuAddDate.csv", encoding='utf-8')	

    # Convert Order_Date string to date and extract relevant date values
    #https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    SD['Order_Date'] = pd.to_datetime(SD['Order_Date'], format= "%d-%b-%y")
    SD['OD_Year'] = SD['Order_Date'].dt.strftime('%Y')
    SD['OD_MonthNum'] = SD['Order_Date'].dt.strftime('%m')
    SD['OD_MonthLab'] = SD['Order_Date'].dt.strftime('%B')
    SD['OD_MonthDay'] = SD['Order_Date'].dt.strftime('%d')
    SD['OD_WeekDay'] = SD['Order_Date'].dt.strftime('%A')
    SD["NetRevenue"]= SD.Total_Net_Price_CAD_ - SD.Total_Refunded_CAD_

    # Filter out any blank orderdate values
    FilterSD =    SD[(SD['Order_Date']!="")]

    #Extract meta data from SD
    ##Identify all unique brands
    AllBrands = SD.attribute_set.unique().astype(str)
    AllBrands = AllBrands[(AllBrands!="Discontinued") & (AllBrands!="nan") ]
    AllSuppliers = SD.Supplier.unique()
    AllBucketValues = SD.Order_Bucket.unique()

    #Group by Attribute set & year sold, then count number of sales
    monthlyGroupingN = (FilterSD.groupby(["attribute_set", "OD_MonthNum", "OD_MonthDay"], as_index=False)
                    ['Order_Date'].
                    agg({"count":"count"}).sort_values(["attribute_set","OD_MonthNum","OD_MonthDay"] , ascending = True))
    monthlyGroupingR = (FilterSD.groupby(["attribute_set", "OD_MonthNum", "OD_MonthDay"], as_index=False)
                    ['NetRevenue'].
                    agg({"NetRevenue":"sum"}).sort_values(["attribute_set","OD_MonthNum", "OD_MonthDay"] , ascending = True))
    monthlyGroupingAll = monthlyGroupingN.merge(monthlyGroupingR, on=('attribute_set', 'OD_MonthNum', "OD_MonthDay")).reset_index(drop=True)


    DueDatePrediction = pd.DataFrame(columns = ['attribute_set',"MNum",'Month' , 'Day', 
                                                    "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", 
                                                    "Sep", "Oct", "Nov", "Dec","TotalOAD","peakDays", "test1"])

    for value in range(len(AllBrands)-1):

        brand = AllBrands[value] 
        print(brand)
        BrandYearSS = monthlyGroupingAll[(monthlyGroupingN['attribute_set']==brand)].reset_index(drop=True)

        BrandYearSS["OverSales"] =  BrandYearSS["count"] - st.mean(BrandYearSS["count"])
        BrandYearSS["OverRev"] = BrandYearSS["NetRevenue"] -  st.mean(BrandYearSS["NetRevenue"]) 

        OverIdentify = BrandYearSS[(BrandYearSS["OverSales"]>0) & (BrandYearSS["OverRev"]>0)]
        OverIdentify_Tally = OverIdentify.groupby(["OD_MonthNum"], as_index=False)['OD_MonthNum'].agg({"OverCounter":"count"})

        OverIdentify_Final = pd.DataFrame({"OD_MonthNum":["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]})
        OverIdentify_Final = OverIdentify_Final.merge(OverIdentify_Tally, on=('OD_MonthNum'), how = "outer").reset_index(drop=True)
        OverIdentify_Final = OverIdentify_Final.fillna(0)

        if len(OverIdentify_Final)>1:

            for m in range(len(OverIdentify_Final)-1):
                Base = (OverIdentify_Final.loc[m+1, "OverCounter"])
                Threshold = (OverIdentify_Final.loc[m, "OverCounter"]*1.5)
                if Base>Threshold:
                    if Base>=6:###Consider reducing to 6
                        OverIdentify_Final.loc[m, "test1"] = "Update"
                    else:
                        OverIdentify_Final.loc[m, "test1"]  = ""
                else:
                    OverIdentify_Final.loc[m, "test1"]  = ""

            FindDup = len(OverIdentify_Final[OverIdentify_Final["test1"]=="Update"])

            if FindDup > 0:
                if FindDup==1:
                    OverIdentify_Final["test2"] = OverIdentify_Final["test1"]

                    WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                    SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)

                elif FindDup>1:
                    MultiUpdate = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"]
                    MaxMonths = (len(OverIdentify_Final)-1 - MultiUpdate.index[-1])+1
                    MultiUpdate["index"] = MultiUpdate.index
                    MultiUpdate = MultiUpdate.reset_index(drop=True)

                    for s in range(len(MultiUpdate)):
                        if s != (len(MultiUpdate)-1):
                            IndexVal = MultiUpdate.loc[s,"index"]
                            MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal), "OverCounter"])
                        else: 
                            IndexVal = MultiUpdate.loc[s,"index"]
                            MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal-1), "OverCounter"], OverIdentify_Final.loc[0, "OverCounter"])                  

                    WinMonth = MultiUpdate.sort_values("Value", ascending = False).reset_index(drop=True).loc[0,"index"]

                    for m in range(len(OverIdentify_Final)-1):
                        if m == WinMonth:
                            OverIdentify_Final.loc[WinMonth, "test2"] = "Update"
                        else:
                            OverIdentify_Final.loc[m, "test2"]  = ""

                    WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                    SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)


                #Determine the end of peak
                count = 0
                sumDays = 0
                dayCount = 0

                for i in range(int(WinningMonth), len(OverIdentify_Final)):
                    if i != 12:
                        month = OverIdentify_Final.loc[i, "OD_MonthNum"]
                        days = OverIdentify_Final.loc[i, "OverCounter"]
                        count = count + 1
                        sumDays = sumDays + days
                        avg = (sumDays/count)*.75

                        if(days >= avg):
                            dayCount = dayCount + 30
                        else:
                            break
                    elif i == 12:
                        month = OverIdentify_Final.loc[0, "OD_MonthNum"]
                        days = OverIdentify_Final.loc[0, "OverCounter"]
                        count = count + 1
                        sumDays = sumDays + days
                        avg = (sumDays/count)*.75

                        if(days >= avg):
                            dayCount = dayCount + 30
                        else:
                            break


                #Determine day of WinningMonth of due date
                TextMonth = calendar.month_name[int(WinningMonth)]
                MaxCountIndex = SubsetWinMonth[SubsetWinMonth["count"] == (SubsetWinMonth["count"].max())].index[0]
                MaxRevIndex = SubsetWinMonth[SubsetWinMonth["NetRevenue"] == (SubsetWinMonth["NetRevenue"].max())].index[0]

                if MaxCountIndex == MaxRevIndex:
                    MaxOfMonth = MaxCountIndex
                    if MaxOfMonth == len(SubsetWinMonth)-1:
                        MinusLast = SubsetWinMonth.loc[0:len(SubsetWinMonth)-2]
                        MaxCountIndex = MinusLast[MinusLast["count"] == (MinusLast["count"].max())].index[0]
                        MaxRevIndex = MinusLast[MinusLast["NetRevenue"] == (MinusLast["NetRevenue"].max())].index[0]
                        if MaxCountIndex == MaxRevIndex:
                            MaxOfMonth = MaxCountIndex
                            MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                        else: 
                            MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                            MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                    else:
                        MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                else: 
                    MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                    MinSubset = SubsetWinMonth.loc[MaxOfMonth:]

                MinCountIndex = MinSubset[MinSubset["count"]==MinSubset.loc[MaxOfMonth:, "count"].min()].index[-1]
                MinRevIndex =   MinSubset[MinSubset["NetRevenue"]==MinSubset.loc[MaxOfMonth:, "NetRevenue"].min()].index[-1]    

                if MinCountIndex == MinRevIndex:
                    FinalIndex = MinRevIndex
                    SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                    WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
                else: 
                    FinalIndex = min(MinCountIndex, MinRevIndex)
                    SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                    WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
            else:
                TextMonth = "None"
                WinningMonth = 0
                WinDay = 0

        #Formatting Data
        AllRecomMon = str()
        AllRecomMonRou = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"].reset_index(drop = True)
        for m in range(len(AllRecomMonRou)):
            AllRecomMon = AllRecomMon + calendar.month_name[int(AllRecomMonRou.loc[m,"OD_MonthNum"])]+ "-"

        OC = OverIdentify_Final["OverCounter"]
        TotalOAD = sum(OC)
        DueDateData = brand,WinningMonth,TextMonth,WinDay,OC[0],OC[1],OC[2],OC[3],OC[4],OC[5],OC[6],OC[7],OC[8],OC[9],OC[10],OC[11],TotalOAD,dayCount, str(AllRecomMon)

        DueDatePrediction = DueDatePrediction.append(pd.Series(DueDateData, index=DueDatePrediction.columns),  ignore_index=True)
        Daydata = DueDatePrediction[["attribute_set","peakDays"]]

    #subset internal sku, attribute set and order date from SD data frame
    SoldSkus = pd.DataFrame(SD.internal_sku.unique().astype(str))
    SoldSkus.columns = ["internal_sku"]

    #Merge SoldSkus with SkuAddDate
    SoldSkus = pd.merge(SoldSkus, SkuAddDate, how='inner', on="internal_sku")
    SoldSkus['add_date'] = pd.to_datetime(SoldSkus['add_date'], format= "%Y-%m-%d")
    BrandLabel = SD[["internal_sku", "attribute_set"]]
    SoldSkus = pd.merge(SoldSkus, BrandLabel, how='inner', on="internal_sku").drop_duplicates(keep=False)
    SoldSkus= SoldSkus.reset_index()

    for i in range(len(SoldSkus)):
        sku = SoldSkus.loc[i,"internal_sku"]
        print(sku)
        Brand = SoldSkus.loc[i,"attribute_set"]
        AddDate = SoldSkus.loc[i,"add_date"]

        AllOrderDates = SD[SD["internal_sku"]==sku].Order_Date.sort_values().reset_index().loc[0,"Order_Date"]
        PeakDayValue = Daydata[Daydata["attribute_set"]==Brand].reset_index().loc[0, "peakDays"]

        RevSubset = SD[SD["internal_sku"]==sku][["Order_Date", "NetRevenue" ]]
        RevSubset["SellDistance"] = abs(RevSubset["Order_Date"]-AddDate).dt.days
        RevSubset["SellDifference"] = PeakDayValue -  RevSubset["SellDistance"]     
        NewSkuRev = sum(RevSubset[RevSubset["SellDifference"] >= 0]["NetRevenue"])

        SoldSkus.loc[i,"FirstSoldDate"] = AllOrderDates
        SoldSkus.loc[i,"PeakDayRange"] = PeakDayValue
        SoldSkus.loc[i,"NewSkuRev"] = NewSkuRev

        sData = str(round((i/len(SoldSkus))*100)), " ", str(SoldSkus.loc[i,"internal_sku"]), " ", str(NewSkuRev)
        status_value.set(sData)


    SoldSkus['DayDiff'] = abs(SoldSkus.FirstSoldDate-SoldSkus.add_date).dt.days


    for i in range(len(SoldSkus)):
        DayDiff = SoldSkus.loc[i,"DayDiff"]
        pdr = SoldSkus.loc[i,"PeakDayRange"]

        if DayDiff <= pdr: 
            SoldSkus.loc[i, "UnderPeakDays"] = 1
        else: 
            SoldSkus.loc[i, "UnderPeakDays"] = 0

        if (DayDiff<365):
            SoldSkus.loc[i, "UnderYear"] = 1
            SoldSkus.loc[i, "OverYear"] = 0

            if(DayDiff<180):
                SoldSkus.loc[i, "HalfYear"] = 1
            else:
                SoldSkus.loc[i, "HalfYear"] = 0
        else:
            SoldSkus.loc[i, "UnderYear"] = 0 
            SoldSkus.loc[i, "OverYear"] = 1        


    AvgDays = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['DayDiff'].
                    agg({"AvgDayDiff":"mean"}).sort_values(["AvgDayDiff"] , ascending = True))

    TimesSold = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['DayDiff'].
                    agg({"TimesSold":"count"}).sort_values(["TimesSold"] , ascending = True))

    UnderYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['UnderYear'].
                    agg({"UnderYear":"sum"}))

    OverYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['OverYear'].
                    agg({"OverYear":"sum"}))

    HalfYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['HalfYear'].
                    agg({"HalfYear":"sum"}))

    UnderPeakDays = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['UnderPeakDays'].
                    agg({"UnderPeakDays":"sum"}))

    NewSkuRev = (SoldSkus.groupby(["attribute_set"], as_index=False)
                    ['NewSkuRev'].
                    agg({"NewSkuRev":"sum"}))

    finalskuCount = AvgDays.merge(TimesSold, on="attribute_set" )
    finalskuCount = finalskuCount.merge(UnderYear, on="attribute_set" )
    finalskuCount = finalskuCount.merge(OverYear, on="attribute_set" )
    finalskuCount = finalskuCount.merge(HalfYear, on="attribute_set" )
    finalskuCount = finalskuCount.merge(UnderPeakDays, on="attribute_set" )
    finalskuCount = finalskuCount.merge(NewSkuRev, on="attribute_set" )
    finalskuCount["%UnderYear"] = finalskuCount["UnderYear"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
    finalskuCount["%HalfYear"] = finalskuCount["HalfYear"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
    finalskuCount["%UnderPeakDays"] = finalskuCount["UnderPeakDays"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
    finalskuCount = finalskuCount.sort_values(["TimesSold", "AvgDayDiff", "UnderYear"] , ascending = [False,True, True])

    #Aquire net revenue for all brands
    BrandRev = monthlyGroupingAll.groupby("attribute_set")["NetRevenue"].agg("sum")
    FinalBrandProfile = finalskuCount.merge(BrandRev, on="attribute_set" ).merge(DueDatePrediction, on="attribute_set" )
    FinalBrandProfile["%NewSkuRev"] = (FinalBrandProfile["NewSkuRev"]/FinalBrandProfile["NetRevenue"])*100
    FinalBrandProfile["%NewSkuRev"] = (FinalBrandProfile["NewSkuRev"]/FinalBrandProfile["NetRevenue"])*100

    UpdatePrioritydf = FinalBrandProfile[["attribute_set", "MNum", "Day", "NewSkuRev", "%NewSkuRev"]]
    UpdatePrioritydf["MNum"] = pd.to_numeric(UpdatePrioritydf["MNum"])
    UpdatePrioritydf["Day"] = pd.to_numeric(UpdatePrioritydf["Day"])
    UpdatePrioritydf = UpdatePrioritydf[(UpdatePrioritydf["NewSkuRev"]>0) & (UpdatePrioritydf["MNum"] != 0)]

    UpdatePrioritydf = UpdatePrioritydf.sort_values(["MNum", "Day", "NewSkuRev"], ascending = [True, True, False]).reset_index()
    #UpdatePrioritydf.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\UpdatePrioritydf.csv', index = None, header=True)

    UpdatePrioritydf.loc[(UpdatePrioritydf["MNum"]==2) & (UpdatePrioritydf["Day"]==29), "Day"] = 28
    UpdatePrioritydf["Year"] = year
    UpdatePrioritydf["MNum"] = UpdatePrioritydf["MNum"].map("{:02}".format)
    UpdatePrioritydf["Day"] = UpdatePrioritydf["Day"].map("{:02}".format)

    UpdatePrioritydf["Year"] = UpdatePrioritydf["Year"].astype(str)
    UpdatePrioritydf["MNum"] = UpdatePrioritydf["MNum"].astype(str)
    UpdatePrioritydf["Day"] = UpdatePrioritydf["Day"].astype(str)

    UpdatePrioritydf["Date"] = UpdatePrioritydf["Year"]+"-"+UpdatePrioritydf["MNum"]+"-"+UpdatePrioritydf["Day"] 
    UpdatePrioritydf["Date"] = pd.to_datetime(UpdatePrioritydf["Date"], format= "%Y-%m-%d")
    UpdatePrioritydf["Weekday"] = UpdatePrioritydf["Date"].dt.dayofweek
    UpdatePrioritydf["WeekNum"] = UpdatePrioritydf["Date"].dt.week
    UpdatePrioritydf["Ref"] = 0

    for w in reversed(range(53)):
        weekBrands = UpdatePrioritydf[UpdatePrioritydf["WeekNum"]==w]
        BrandPerWeek = len(weekBrands)
        #print(BrandPerWeek)
        if(BrandPerWeek > updateLimit):
            BrandDiff = BrandPerWeek - updateLimit
            MovingBrandList = (list(weekBrands.sort_values("NewSkuRev", ascending = True)
                                    .reset_index(drop=True)[0:BrandDiff]["attribute_set"]))
            UpdatePrioritydf.loc[UpdatePrioritydf["attribute_set"].isin(MovingBrandList), "WeekNum"] = w-1
        weekBrands = UpdatePrioritydf[UpdatePrioritydf["WeekNum"]==w].sort_values("NewSkuRev", ascending = False).reset_index()
        for f in range(len(weekBrands)):
            brand = weekBrands["attribute_set"][f]
            UpdatePrioritydf.loc[UpdatePrioritydf["attribute_set"]==brand, "Ref"] = f

    UpdatePrioritydf["WeekNum"] = UpdatePrioritydf["WeekNum"].map("{:02}".format)

    UpdateSchedule = pd.DataFrame(data={'Day': 1, 'WeekNum': range(0,53), 'Year': year})
    UpdateSchedule["Day"] = UpdateSchedule["Day"].astype(str)
    UpdateSchedule["WeekNum"] = UpdateSchedule["WeekNum"].map("{:02}".format).astype(str)
    UpdateSchedule["Year"] = UpdateSchedule["Year"].astype(str)

    UpdateSchedule["Date_temp"] = UpdateSchedule["Day"]+"-"+UpdateSchedule["WeekNum"]+"-"+UpdateSchedule["Year"] 
    UpdateSchedule["Date_temp"] = pd.to_datetime(UpdateSchedule["Date_temp"], format= "%w-%W-%Y")
    UpdateSchedule["Month"] = UpdateSchedule["Date_temp"].dt.month_name()
    UpdateSchedule["Date"] = UpdateSchedule["Date_temp"].dt.day

    RankedBrands = UpdatePrioritydf.pivot(index='WeekNum', columns='Ref', values='attribute_set')

    UpdateScheduleFinal = (UpdateSchedule
                           .merge(RankedBrands, on=('WeekNum'))
                           .drop(columns=["Day","WeekNum","Year","Date_temp"])
                           .fillna(""))

    return(UpdateScheduleFinal)
            
        

def Resetfun():
    #Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)

def Close():
    root.destroy()

def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
status_value = StringVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
Frame_1 = Frame(root)
Frame_1 .pack()
Frame_2 = Frame(root)
Frame_2 .pack()
Frame_3 = Frame(root)
Frame_3 .pack()
Frame_4 = Frame(root)
Frame_4 .pack()
Frame_5 = Frame(root)
Frame_5 .pack()

get_year = Spinbox(Frame_1, from_=2000, to=2100, width = 4, values = 2019)
get_wLimit = Spinbox(Frame_1, from_=1, to=100, width = 2, values = 4)
get_dThresh = Spinbox(Frame_1, from_=1, to=99, width = 2, values = 6)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(Frame_1, text="Parameters:").pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(Frame_2, text="                    ").pack(side = LEFT)

Label(Frame_2, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(Frame_2, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(Frame_2, text="*50%*", variable=get_perChange, value=50)
R2.select()
R2.pack(side = LEFT)

R3 = Radiobutton(Frame_2, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(Frame_2, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

PathSelect = Button(Frame_3, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(Frame_3, text=Path,  width = 45, anchor="w")
SelectedPath.pack(side = LEFT)

RunPrediction = Button(Frame_4, text ="Execute", state="normal", command = UpdatePredictor)
RunPrediction.pack(side = LEFT)

Status = Label(Frame_4, textvariable=status_value, width = 42, anchor=W, justify=LEFT)
Status.pack(side = LEFT)

#Reset = Button(Frame_4, text ="Stop", state="disable", command = Resetfun)
#Reset.pack(side = LEFT)


root.title(str(Version+" "+Title +" by "+Author+" ("+Date+")"))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()

In [ ]:
import threading 
import time 
  
def UpdatePredictor(): 
    while True: 
        
        print('thread running') 
        
        global GlobalStatus 
        if GlobalStatus: 
            break
  
GlobalStatus = False
t1 = threading.Thread(target = UpdatePredictor) 
t1.start() 
time.sleep(1) 
GlobalStatus = True
t1.join() 
print('thread killed') 

In [ ]:
#combining GUI and algorithm with deafult setting set

#Import Necessary Packages
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading


#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"


def UpdatePredictor():
    import pandas as pd
    import numpy as np
    import statistics as st
    import calendar
    import datetime
    import re
    import matplotlib.pyplot as plt
    pd.options.mode.chained_assignment = None
    
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    if GlobalStatus == "normal":
        
        year = int(get_year.get())
        wLimit = int(get_wLimit.get())
        dThresh = int(get_dThresh.get())
        perChange = int(get_perChange.get())
        Reset.config(state="normal")
        RunPrediction.config(state="disabled")
        PathSelect.config(state="disabled")
        get_year.config(state="disabled")
        get_wLimit.config(state="disabled")
        get_dThresh.config(state="disabled")
        R1.config(state="disabled")
        R2.config(state="disabled")
        R3.config(state="disabled")
        R4.config(state="disabled")

        ## Loading Data
        PastYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/PastYearDataNew.csv", encoding='utf-8')	
        CurrentYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/CurrentYearDataNew.csv", encoding='utf-8')	
        #Combine Past and recent years sales data
        SD = pd.concat([PastYearData, CurrentYearData], ignore_index=True)
        SDnum = pd.concat([PastYearData, CurrentYearData], ignore_index=True)
        SkuAddDate =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/skuAddDate.csv", encoding='utf-8')	

        # Convert Order_Date string to date and extract relevant date values
        #https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
        SD['Order_Date'] = pd.to_datetime(SD['Order_Date'], format= "%d-%b-%y")
        SD['OD_Year'] = SD['Order_Date'].dt.strftime('%Y')
        SD['OD_MonthNum'] = SD['Order_Date'].dt.strftime('%m')
        SD['OD_MonthLab'] = SD['Order_Date'].dt.strftime('%B')
        SD['OD_MonthDay'] = SD['Order_Date'].dt.strftime('%d')
        SD['OD_WeekDay'] = SD['Order_Date'].dt.strftime('%A')
        SD["NetRevenue"]= SD.Total_Net_Price_CAD_ - SD.Total_Refunded_CAD_

        # Filter out any blank orderdate values
        FilterSD =    SD[(SD['Order_Date']!="")]

        #Extract meta data from SD
        ##Identify all unique brands
        AllBrands = SD.attribute_set.unique().astype(str)
        AllBrands = AllBrands[(AllBrands!="Discontinued") & (AllBrands!="nan") ]
        AllSuppliers = SD.Supplier.unique()
        AllBucketValues = SD.Order_Bucket.unique()

        #Group by Attribute set & year sold, then count number of sales
        monthlyGroupingN = (FilterSD.groupby(["attribute_set", "OD_MonthNum", "OD_MonthDay"], as_index=False)
                        ['Order_Date'].
                        agg({"count":"count"}).sort_values(["attribute_set","OD_MonthNum","OD_MonthDay"] , ascending = True))
        monthlyGroupingR = (FilterSD.groupby(["attribute_set", "OD_MonthNum", "OD_MonthDay"], as_index=False)
                        ['NetRevenue'].
                        agg({"NetRevenue":"sum"}).sort_values(["attribute_set","OD_MonthNum", "OD_MonthDay"] , ascending = True))
        monthlyGroupingAll = monthlyGroupingN.merge(monthlyGroupingR, on=('attribute_set', 'OD_MonthNum', "OD_MonthDay")).reset_index(drop=True)


        DueDatePrediction = pd.DataFrame(columns = ['attribute_set',"MNum",'Month' , 'Day', 
                                                        "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", 
                                                        "Sep", "Oct", "Nov", "Dec","TotalOAD","peakDays", "test1"])

        for value in range(len(AllBrands)-1):

            brand = AllBrands[value] 
            BrandYearSS = monthlyGroupingAll[(monthlyGroupingN['attribute_set']==brand)].reset_index(drop=True)

            BrandYearSS["OverSales"] =  BrandYearSS["count"] - st.mean(BrandYearSS["count"])
            BrandYearSS["OverRev"] = BrandYearSS["NetRevenue"] -  st.mean(BrandYearSS["NetRevenue"]) 

            OverIdentify = BrandYearSS[(BrandYearSS["OverSales"]>0) & (BrandYearSS["OverRev"]>0)]
            OverIdentify_Tally = OverIdentify.groupby(["OD_MonthNum"], as_index=False)['OD_MonthNum'].agg({"OverCounter":"count"})

            OverIdentify_Final = pd.DataFrame({"OD_MonthNum":["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]})
            OverIdentify_Final = OverIdentify_Final.merge(OverIdentify_Tally, on=('OD_MonthNum'), how = "outer").reset_index(drop=True)
            OverIdentify_Final = OverIdentify_Final.fillna(0)

            if len(OverIdentify_Final)>1:

                for m in range(len(OverIdentify_Final)-1):
                    Base = (OverIdentify_Final.loc[m+1, "OverCounter"])
                    Threshold = (OverIdentify_Final.loc[m, "OverCounter"]*1.5)
                    if Base>Threshold:
                        if Base>=6:###Consider reducing to 6
                            OverIdentify_Final.loc[m, "test1"] = "Update"
                        else:
                            OverIdentify_Final.loc[m, "test1"]  = ""
                    else:
                        OverIdentify_Final.loc[m, "test1"]  = ""

                FindDup = len(OverIdentify_Final[OverIdentify_Final["test1"]=="Update"])

                if FindDup > 0:
                    if FindDup==1:
                        OverIdentify_Final["test2"] = OverIdentify_Final["test1"]

                        WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                        SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)

                    elif FindDup>1:
                        MultiUpdate = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"]
                        MaxMonths = (len(OverIdentify_Final)-1 - MultiUpdate.index[-1])+1
                        MultiUpdate["index"] = MultiUpdate.index
                        MultiUpdate = MultiUpdate.reset_index(drop=True)

                        for s in range(len(MultiUpdate)):
                            if s != (len(MultiUpdate)-1):
                                IndexVal = MultiUpdate.loc[s,"index"]
                                MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal), "OverCounter"])
                            else: 
                                IndexVal = MultiUpdate.loc[s,"index"]
                                MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal-1), "OverCounter"], OverIdentify_Final.loc[0, "OverCounter"])                  

                        WinMonth = MultiUpdate.sort_values("Value", ascending = False).reset_index(drop=True).loc[0,"index"]

                        for m in range(len(OverIdentify_Final)-1):
                            if m == WinMonth:
                                OverIdentify_Final.loc[WinMonth, "test2"] = "Update"
                            else:
                                OverIdentify_Final.loc[m, "test2"]  = ""

                        WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                        SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)


                    #Determine the end of peak
                    count = 0
                    sumDays = 0
                    dayCount = 0

                    for i in range(int(WinningMonth), len(OverIdentify_Final)):
                        if i != 12:
                            month = OverIdentify_Final.loc[i, "OD_MonthNum"]
                            days = OverIdentify_Final.loc[i, "OverCounter"]
                            count = count + 1
                            sumDays = sumDays + days
                            avg = (sumDays/count)*.75

                            if(days >= avg):
                                dayCount = dayCount + 30
                            else:
                                break
                        elif i == 12:
                            month = OverIdentify_Final.loc[0, "OD_MonthNum"]
                            days = OverIdentify_Final.loc[0, "OverCounter"]
                            count = count + 1
                            sumDays = sumDays + days
                            avg = (sumDays/count)*.75

                            if(days >= avg):
                                dayCount = dayCount + 30
                            else:
                                break


                    #Determine day of WinningMonth of due date
                    TextMonth = calendar.month_name[int(WinningMonth)]
                    MaxCountIndex = SubsetWinMonth[SubsetWinMonth["count"] == (SubsetWinMonth["count"].max())].index[0]
                    MaxRevIndex = SubsetWinMonth[SubsetWinMonth["NetRevenue"] == (SubsetWinMonth["NetRevenue"].max())].index[0]

                    if MaxCountIndex == MaxRevIndex:
                        MaxOfMonth = MaxCountIndex
                        if MaxOfMonth == len(SubsetWinMonth)-1:
                            MinusLast = SubsetWinMonth.loc[0:len(SubsetWinMonth)-2]
                            MaxCountIndex = MinusLast[MinusLast["count"] == (MinusLast["count"].max())].index[0]
                            MaxRevIndex = MinusLast[MinusLast["NetRevenue"] == (MinusLast["NetRevenue"].max())].index[0]
                            if MaxCountIndex == MaxRevIndex:
                                MaxOfMonth = MaxCountIndex
                                MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                            else: 
                                MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                                MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                        else:
                            MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                    else: 
                        MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                        MinSubset = SubsetWinMonth.loc[MaxOfMonth:]

                    MinCountIndex = MinSubset[MinSubset["count"]==MinSubset.loc[MaxOfMonth:, "count"].min()].index[-1]
                    MinRevIndex =   MinSubset[MinSubset["NetRevenue"]==MinSubset.loc[MaxOfMonth:, "NetRevenue"].min()].index[-1]    

                    if MinCountIndex == MinRevIndex:
                        FinalIndex = MinRevIndex
                        SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                        WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
                    else: 
                        FinalIndex = min(MinCountIndex, MinRevIndex)
                        SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                        WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
                else:
                    TextMonth = "None"
                    WinningMonth = 0
                    WinDay = 0

            #Formatting Data
            AllRecomMon = str()
            AllRecomMonRou = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"].reset_index(drop = True)
            for m in range(len(AllRecomMonRou)):
                AllRecomMon = AllRecomMon + calendar.month_name[int(AllRecomMonRou.loc[m,"OD_MonthNum"])]+ "-"

            OC = OverIdentify_Final["OverCounter"]
            TotalOAD = sum(OC)
            DueDateData = brand,WinningMonth,TextMonth,WinDay,OC[0],OC[1],OC[2],OC[3],OC[4],OC[5],OC[6],OC[7],OC[8],OC[9],OC[10],OC[11],TotalOAD,dayCount, str(AllRecomMon)

            DueDatePrediction = DueDatePrediction.append(pd.Series(DueDateData, index=DueDatePrediction.columns),  ignore_index=True)
            Daydata = DueDatePrediction[["attribute_set","peakDays"]]

        #subset internal sku, attribute set and order date from SD data frame
        SoldSkus = pd.DataFrame(SD.internal_sku.unique().astype(str))
        SoldSkus.columns = ["internal_sku"]

        #Merge SoldSkus with SkuAddDate
        SoldSkus = pd.merge(SoldSkus, SkuAddDate, how='inner', on="internal_sku")
        SoldSkus['add_date'] = pd.to_datetime(SoldSkus['add_date'], format= "%Y-%m-%d")
        BrandLabel = SD[["internal_sku", "attribute_set"]]
        SoldSkus = pd.merge(SoldSkus, BrandLabel, how='inner', on="internal_sku").drop_duplicates(keep=False)
        SoldSkus= SoldSkus.reset_index()

        for i in range(len(SoldSkus)):
            if GlobalStatus == "normal":
                sku = SoldSkus.loc[i,"internal_sku"]
                Brand = SoldSkus.loc[i,"attribute_set"]
                AddDate = SoldSkus.loc[i,"add_date"]

                AllOrderDates = SD[SD["internal_sku"]==sku].Order_Date.sort_values().reset_index().loc[0,"Order_Date"]
                PeakDayValue = Daydata[Daydata["attribute_set"]==Brand].reset_index().loc[0, "peakDays"]

                RevSubset = SD[SD["internal_sku"]==sku][["Order_Date", "NetRevenue" ]]
                RevSubset["SellDistance"] = abs(RevSubset["Order_Date"]-AddDate).dt.days
                RevSubset["SellDifference"] = PeakDayValue -  RevSubset["SellDistance"]     
                NewSkuRev = sum(RevSubset[RevSubset["SellDifference"] >= 0]["NetRevenue"])

                SoldSkus.loc[i,"FirstSoldDate"] = AllOrderDates
                SoldSkus.loc[i,"PeakDayRange"] = PeakDayValue
                SoldSkus.loc[i,"NewSkuRev"] = NewSkuRev

                sData = str(round((i/len(SoldSkus))*100)), " ", str(SoldSkus.loc[i,"internal_sku"]), " ", str(NewSkuRev)
                status_value.set(sData)
            else:
                break
        
        #if GlobalStatus == "normal":
        SoldSkus['DayDiff'] = abs(SoldSkus.FirstSoldDate-SoldSkus.add_date).dt.days


        for i in range(len(SoldSkus)):
            DayDiff = SoldSkus.loc[i,"DayDiff"]
            pdr = SoldSkus.loc[i,"PeakDayRange"]

            if DayDiff <= pdr: 
                SoldSkus.loc[i, "UnderPeakDays"] = 1
            else: 
                SoldSkus.loc[i, "UnderPeakDays"] = 0

            if (DayDiff<365):
                SoldSkus.loc[i, "UnderYear"] = 1
                SoldSkus.loc[i, "OverYear"] = 0

                if(DayDiff<180):
                    SoldSkus.loc[i, "HalfYear"] = 1
                else:
                    SoldSkus.loc[i, "HalfYear"] = 0
            else:
                SoldSkus.loc[i, "UnderYear"] = 0 
                SoldSkus.loc[i, "OverYear"] = 1        


        AvgDays = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['DayDiff'].
                        agg({"AvgDayDiff":"mean"}).sort_values(["AvgDayDiff"] , ascending = True))

        TimesSold = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['DayDiff'].
                        agg({"TimesSold":"count"}).sort_values(["TimesSold"] , ascending = True))

        UnderYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['UnderYear'].
                        agg({"UnderYear":"sum"}))

        OverYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['OverYear'].
                        agg({"OverYear":"sum"}))

        HalfYear = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['HalfYear'].
                        agg({"HalfYear":"sum"}))

        UnderPeakDays = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['UnderPeakDays'].
                        agg({"UnderPeakDays":"sum"}))

        NewSkuRev = (SoldSkus.groupby(["attribute_set"], as_index=False)
                        ['NewSkuRev'].
                        agg({"NewSkuRev":"sum"}))

        finalskuCount = AvgDays.merge(TimesSold, on="attribute_set" )
        finalskuCount = finalskuCount.merge(UnderYear, on="attribute_set" )
        finalskuCount = finalskuCount.merge(OverYear, on="attribute_set" )
        finalskuCount = finalskuCount.merge(HalfYear, on="attribute_set" )
        finalskuCount = finalskuCount.merge(UnderPeakDays, on="attribute_set" )
        finalskuCount = finalskuCount.merge(NewSkuRev, on="attribute_set" )
        finalskuCount["%UnderYear"] = finalskuCount["UnderYear"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
        finalskuCount["%HalfYear"] = finalskuCount["HalfYear"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
        finalskuCount["%UnderPeakDays"] = finalskuCount["UnderPeakDays"] /(finalskuCount["OverYear"] + finalskuCount["UnderYear"])
        finalskuCount = finalskuCount.sort_values(["TimesSold", "AvgDayDiff", "UnderYear"] , ascending = [False,True, True])

        #Aquire net revenue for all brands
        BrandRev = monthlyGroupingAll.groupby("attribute_set")["NetRevenue"].agg("sum")
        FinalBrandProfile = finalskuCount.merge(BrandRev, on="attribute_set" ).merge(DueDatePrediction, on="attribute_set" )
        FinalBrandProfile["%NewSkuRev"] = (FinalBrandProfile["NewSkuRev"]/FinalBrandProfile["NetRevenue"])*100
        FinalBrandProfile["%NewSkuRev"] = (FinalBrandProfile["NewSkuRev"]/FinalBrandProfile["NetRevenue"])*100

        UpdatePrioritydf = FinalBrandProfile[["attribute_set", "MNum", "Day", "NewSkuRev", "%NewSkuRev"]]
        UpdatePrioritydf["MNum"] = pd.to_numeric(UpdatePrioritydf["MNum"])
        UpdatePrioritydf["Day"] = pd.to_numeric(UpdatePrioritydf["Day"])
        UpdatePrioritydf = UpdatePrioritydf[(UpdatePrioritydf["NewSkuRev"]>0) & (UpdatePrioritydf["MNum"] != 0)]

        UpdatePrioritydf = UpdatePrioritydf.sort_values(["MNum", "Day", "NewSkuRev"], ascending = [True, True, False]).reset_index()
        #UpdatePrioritydf.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\UpdatePrioritydf.csv', index = None, header=True)

        UpdatePrioritydf.loc[(UpdatePrioritydf["MNum"]==2) & (UpdatePrioritydf["Day"]==29), "Day"] = 28
        UpdatePrioritydf["Year"] = year
        UpdatePrioritydf["MNum"] = UpdatePrioritydf["MNum"].map("{:02}".format)
        UpdatePrioritydf["Day"] = UpdatePrioritydf["Day"].map("{:02}".format)

        UpdatePrioritydf["Year"] = UpdatePrioritydf["Year"].astype(str)
        UpdatePrioritydf["MNum"] = UpdatePrioritydf["MNum"].astype(str)
        UpdatePrioritydf["Day"] = UpdatePrioritydf["Day"].astype(str)

        UpdatePrioritydf["Date"] = UpdatePrioritydf["Year"]+"-"+UpdatePrioritydf["MNum"]+"-"+UpdatePrioritydf["Day"] 
        UpdatePrioritydf["Date"] = pd.to_datetime(UpdatePrioritydf["Date"], format= "%Y-%m-%d")
        UpdatePrioritydf["Weekday"] = UpdatePrioritydf["Date"].dt.dayofweek
        UpdatePrioritydf["WeekNum"] = UpdatePrioritydf["Date"].dt.week
        UpdatePrioritydf["Ref"] = 0

        for w in reversed(range(53)):
            weekBrands = UpdatePrioritydf[UpdatePrioritydf["WeekNum"]==w]
            BrandPerWeek = len(weekBrands)
            #print(BrandPerWeek)
            if(BrandPerWeek > updateLimit):
                BrandDiff = BrandPerWeek - updateLimit
                MovingBrandList = (list(weekBrands.sort_values("NewSkuRev", ascending = True)
                                        .reset_index(drop=True)[0:BrandDiff]["attribute_set"]))
                UpdatePrioritydf.loc[UpdatePrioritydf["attribute_set"].isin(MovingBrandList), "WeekNum"] = w-1
            weekBrands = UpdatePrioritydf[UpdatePrioritydf["WeekNum"]==w].sort_values("NewSkuRev", ascending = False).reset_index()
            for f in range(len(weekBrands)):
                brand = weekBrands["attribute_set"][f]
                UpdatePrioritydf.loc[UpdatePrioritydf["attribute_set"]==brand, "Ref"] = f

        UpdatePrioritydf["WeekNum"] = UpdatePrioritydf["WeekNum"].map("{:02}".format)

        UpdateSchedule = pd.DataFrame(data={'Day': 1, 'WeekNum': range(0,53), 'Year': year})
        UpdateSchedule["Day"] = UpdateSchedule["Day"].astype(str)
        UpdateSchedule["WeekNum"] = UpdateSchedule["WeekNum"].map("{:02}".format).astype(str)
        UpdateSchedule["Year"] = UpdateSchedule["Year"].astype(str)

        UpdateSchedule["Date_temp"] = UpdateSchedule["Day"]+"-"+UpdateSchedule["WeekNum"]+"-"+UpdateSchedule["Year"] 
        UpdateSchedule["Date_temp"] = pd.to_datetime(UpdateSchedule["Date_temp"], format= "%w-%W-%Y")
        UpdateSchedule["Month"] = UpdateSchedule["Date_temp"].dt.month_name()
        UpdateSchedule["Date"] = UpdateSchedule["Date_temp"].dt.day

        RankedBrands = UpdatePrioritydf.pivot(index='WeekNum', columns='Ref', values='attribute_set')

        UpdateScheduleFinal = (UpdateSchedule
                               .merge(RankedBrands, on=('WeekNum'))
                               .drop(columns=["Day","WeekNum","Year","Date_temp"])
                               .fillna(""))

        return(UpdateScheduleFinal)


        
def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()

def Resetfun():
    global GlobalStatus
    GlobalStatus = "disable"
    Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    root.destroy()

def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
status_value = StringVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
Frame_1 = Frame(root)
Frame_1 .pack()
Frame_2 = Frame(root)
Frame_2 .pack()
Frame_3 = Frame(root)
Frame_3 .pack()
Frame_4 = Frame(root)
Frame_4 .pack()
Frame_5 = Frame(root)
Frame_5 .pack()

get_year = Spinbox(Frame_1, from_=2000, to=2100, width = 4, values = 2019)
get_wLimit = Spinbox(Frame_1, from_=1, to=100, width = 2, values = 4)
get_dThresh = Spinbox(Frame_1, from_=1, to=99, width = 2, values = 6)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(Frame_1, text="Parameters:").pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(Frame_2, text="                    ").pack(side = LEFT)

Label(Frame_2, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(Frame_2, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(Frame_2, text="*50%*", variable=get_perChange, value=50)
R2.select()
R2.pack(side = LEFT)

R3 = Radiobutton(Frame_2, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(Frame_2, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

PathSelect = Button(Frame_3, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(Frame_3, text=Path,  width = 45, anchor="w")
SelectedPath.pack(side = LEFT)

RunPrediction = Button(Frame_4, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

Status = Label(Frame_4, textvariable=status_value, width = 42, anchor=W, justify=LEFT)
Status.pack(side = LEFT)

Reset = Button(Frame_4, text ="Stop", state="disable", command = Resetfun)
Reset.pack(side = LEFT)


root.title(str(Version+" "+Title +" by "+Author+" ("+Date+")"))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()

In [ ]:
#switch status bar to text updates
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading

#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"

def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    Reset.config(state="normal")
    RunPrediction.config(state="disabled")
    PathSelect.config(state="disabled")
    get_year.config(state="disabled")
    get_wLimit.config(state="disabled")
    get_dThresh.config(state="disabled")
    R1.config(state="disabled")
    R2.config(state="disabled")
    R3.config(state="disabled")
    R4.config(state="disabled")
    
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    for i in range(1,100):
        if GlobalStatus == "normal":
            count = i
            print(count)
            time.sleep(1)
            sData = str((i, " ", i/100))
            status_value.set(sData)
            
        else:
            break

    selection = (year, wLimit, dThresh, perChange, Path)
    print(selection)
    process = 0
    
def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()

def Resetfun():
    global GlobalStatus
    GlobalStatus = "disable"
    Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    root.destroy()

def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
status_value = StringVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
Frame_1 = Frame(root)
Frame_1 .pack()
Frame_2 = Frame(root)
Frame_2 .pack()
Frame_3 = Frame(root)
Frame_3 .pack()
Frame_4 = Frame(root)
Frame_4 .pack()
Frame_5 = Frame(root)
Frame_5 .pack()

get_year = Spinbox(Frame_1, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(Frame_1, from_=1, to=100, width = 2)
get_dThresh = Spinbox(Frame_1, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(Frame_1, text="Parameters:").pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(Frame_2, text="                    ").pack(side = LEFT)

Label(Frame_2, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(Frame_2, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(Frame_2, text="*50%*", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(Frame_2, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(Frame_2, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

PathSelect = Button(Frame_3, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(Frame_3, text=Path,  width = 45, anchor="w")
SelectedPath.pack(side = LEFT)

RunPrediction = Button(Frame_4, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

Status = Label(Frame_4, textvariable=status_value, width = 42, anchor=W, justify=LEFT)
Status.pack(side = LEFT)

Reset = Button(Frame_4, text ="Stop", state="disable", command = Resetfun)
Reset.pack(side = LEFT)


root.title(str(Version+" "+Title +" by "+Author+" ("+Date+")"))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()

In [ ]:
sData = str((str(i), " ", str(i/100)))
print(sData)



In [ ]:
#Enable progress bar feature
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading

#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"

def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    Reset.config(state="normal")
    RunPrediction.config(state="disabled")
    PathSelect.config(state="disabled")
    get_year.config(state="disabled")
    get_wLimit.config(state="disabled")
    get_dThresh.config(state="disabled")
    R1.config(state="disabled")
    R2.config(state="disabled")
    R3.config(state="disabled")
    R4.config(state="disabled")
    
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    for i in range(1,100):
        if GlobalStatus == "normal":
            count = i
            print(count)
            time.sleep(1)
            P['value'] = i
            
        else:
            break

    selection = (year, wLimit, dThresh, perChange, Path)
    print(selection)
    process = 0
    
def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()

def Resetfun():
    global GlobalStatus
    GlobalStatus = "disable"
    Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    root.destroy()

def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
Frame_1 = Frame(root)
Frame_1 .pack()
Frame_2 = Frame(root)
Frame_2 .pack()
Frame_3 = Frame(root)
Frame_3 .pack()
Frame_4 = Frame(root)
Frame_4 .pack()
Frame_5 = Frame(root)
Frame_5 .pack()

get_year = Spinbox(Frame_1, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(Frame_1, from_=1, to=100, width = 2)
get_dThresh = Spinbox(Frame_1, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(Frame_1, text="Parameters:").pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(Frame_2, text="                    ").pack(side = LEFT)

Label(Frame_2, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(Frame_2, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(Frame_2, text="*50%*", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(Frame_2, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(Frame_2, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

PathSelect = Button(Frame_3, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(Frame_3, text=Path,  width = 45, anchor="w")
SelectedPath.pack(side = LEFT)

RunPrediction = Button(Frame_4, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

P = ttk.Progressbar(Frame_4, orient="horizontal", length=300, maximum=len(range(1,100))-1, mode="determinate")
P.pack(side = LEFT)

Reset = Button(Frame_4, text ="Stop", state="disable", command = Resetfun)
Reset.pack(side = LEFT)


root.title(str(Version+" "+Title +" by "+Author+" ("+Date+")"))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()

In [ ]:
from tkinter import ttk
from tkinter import *
import time
import threading
    
def UpdatePredictor():
    for i in range(1,12):
        count = i
        print(i)
        time.sleep(1)
        P['value'] = i



def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor ,daemon=True).start()
    #t2 = threading.Thread(target=update ,daemon=True).start()
    
root = Tk()

P = ttk.Progressbar(root, orient="horizontal", length=300, maximum=len(range(1,12))-1, mode="determinate")
P.pack()

L = Label(root, text="In Process")
L.pack()

RunPrediction = Button(root, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

process = 0


root.mainloop()

In [ ]:
#Restructure GUI layout 
#Parameters at top, Path selection second, excution and stop third, progress bar last.
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading


#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"



def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    Reset.config(state="normal")
    RunPrediction.config(state="disabled")
    PathSelect.config(state="disabled")
    get_year.config(state="disabled")
    get_wLimit.config(state="disabled")
    get_dThresh.config(state="disabled")
    R1.config(state="disabled")
    R2.config(state="disabled")
    R3.config(state="disabled")
    R4.config(state="disabled")
    
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    for i in range(1,100):
        if GlobalStatus == "normal":
            count = i
            print(count)
            time.sleep(1)
        else:
            break

    
    selection = (year, wLimit, dThresh, perChange, Path)
    print(selection)
    process = 0
    
def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()

    
def Resetfun():
    global GlobalStatus
    GlobalStatus = "disable"
    Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)
    
def update(): #Reference: https://stackoverflow.com/questions/55760066/stop-a-progress-bar-at-max-value-tkinter
    global process
    P['value'] = process
    if P['value'] >= P['maximum']:
        popup_showinfo()
        return  #EndForLoop
    root.after( 100, update )

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    root.destroy()

    
def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
Frame_1 = Frame(root)
Frame_1 .pack()
Frame_2 = Frame(root)
Frame_2 .pack()
Frame_3 = Frame(root)
Frame_3 .pack()
Frame_4 = Frame(root)
Frame_4 .pack()
Frame_5 = Frame(root)
Frame_5 .pack()

get_year = Spinbox(Frame_1, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(Frame_1, from_=1, to=100, width = 2)
get_dThresh = Spinbox(Frame_1, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(Frame_1, text="Parameters:").pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(Frame_2, text="                    ").pack(side = LEFT)

Label(Frame_2, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(Frame_2, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(Frame_2, text="*50%*", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(Frame_2, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(Frame_2, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

PathSelect = Button(Frame_3, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(Frame_3, text=Path,  width = 45, anchor="w")
SelectedPath.pack(side = LEFT)

RunPrediction = Button(Frame_4, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

P = ttk.Progressbar(Frame_4, orient="horizontal", length=300, maximum=100, mode="determinate")
P.pack(side = LEFT)

Reset = Button(Frame_4, text ="Stop", state="disable", command = Resetfun)
Reset.pack(side = LEFT)


root.title(str(Version+" "+Title +" by "+Author+" ("+Date+")"))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()

In [ ]:
#Restructure GUI layout 
#Parameters at top, Path selection second, excution and stop third, progress bar last.
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading


#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"



def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    Reset.config(state="normal")
    RunPrediction.config(state="disabled")
    PathSelect.config(state="disabled")
    get_year.config(state="disabled")
    get_wLimit.config(state="disabled")
    get_dThresh.config(state="disabled")
    R1.config(state="disabled")
    R2.config(state="disabled")
    R3.config(state="disabled")
    R4.config(state="disabled")
    
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    for i in range(1,100):
        if GlobalStatus == "normal":
            count = i
            print(count)
            time.sleep(1)
        else:
            break

    
    selection = (year, wLimit, dThresh, perChange, Path)
    print(selection)
    process = 0
    
def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()

    
def Resetfun():
    global GlobalStatus
    GlobalStatus = "disable"
    Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)
    
def update(): #Reference: https://stackoverflow.com/questions/55760066/stop-a-progress-bar-at-max-value-tkinter
    global process
    P['value'] = process
    if P['value'] >= P['maximum']:
        popup_showinfo()
        return  #EndForLoop
    root.after( 100, update )

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    root.destroy()

    
def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
Frame_1 = Frame(root)
Frame_1 .pack()
Frame_2 = Frame(root)
Frame_2 .pack()
Frame_3 = Frame(root)
Frame_3 .pack()
Frame_4 = Frame(root)
Frame_4 .pack()
Frame_5 = Frame(root)
Frame_5 .pack()

get_year = Spinbox(Frame_1, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(Frame_1, from_=1, to=100, width = 2)
get_dThresh = Spinbox(Frame_1, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(Frame_1, text="Parameters:").pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_1, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(Frame_1, text=" ").pack(side = LEFT)

Label(Frame_2, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(Frame_2, text="25%", variable=get_perChange, value=25) 
R1.pack(side = LEFT)

R2 = Radiobutton(Frame_2, text="*50%", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(Frame_2, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(Frame_2, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

PathSelect = Button(Frame_3, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(Frame_3, text=Path)
SelectedPath.pack(side = LEFT)

RunPrediction = Button(Frame_4, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

Reset = Button(Frame_4, text ="Stop", state="disable", command = Resetfun)
Reset.pack(side = LEFT)



P = ttk.Progressbar(Frame_5, orient="horizontal", length=300, maximum=100, mode="determinate")
P.pack()


root.title(str(Version+" "+Title +" by "+Author+" ("+Date+")"))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()




In [ ]:
#Integrate Disabling of button and reset button
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time
import threading


#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"



def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    Reset.config(state="normal")
    RunPrediction.config(state="disabled")
    PathSelect.config(state="disabled")
    get_year.config(state="disabled")
    get_wLimit.config(state="disabled")
    get_dThresh.config(state="disabled")
    R1.config(state="disabled")
    R2.config(state="disabled")
    R3.config(state="disabled")
    R4.config(state="disabled")
    
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    for i in range(1,100):
        if GlobalStatus == "normal":
            count = i
            print(count)
            time.sleep(1)
        else:
            break

    
    selection = (year, wLimit, dThresh, perChange, Path)
    print(selection)
    process = 0
    
def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()

    
def Resetfun():
    global GlobalStatus
    GlobalStatus = "disable"
    Reset.config(state="disabled")
    RunPrediction.config(state="normal")
    PathSelect.config(state="normal")
    get_year.config(state="normal")
    get_wLimit.config(state="normal")
    get_dThresh.config(state="normal")
    R1.config(state="normal")
    R2.config(state="normal")
    R3.config(state="normal")
    R4.config(state="normal")

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)
    
def update(): #Reference: https://stackoverflow.com/questions/55760066/stop-a-progress-bar-at-max-value-tkinter
    global process
    P['value'] = process
    if P['value'] >= P['maximum']:
        popup_showinfo()
        return  #EndForLoop
    root.after( 100, update )

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    root.destroy()

    
def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
GlobalStatus = "Normal"
Path = 'C:/'

#Make the frames
TopFrame = Frame(root)
TopFrame.pack()

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

#GUI Layout
Label(TopFrame, text="Parameters:").pack(side = LEFT)

Label(TopFrame, text=" ").pack(side = LEFT)

Label(TopFrame, text="Year:").pack(side = LEFT)
get_year.pack(side = LEFT)

Label(TopFrame, text=" ").pack(side = LEFT)

Label(TopFrame, text="Weekly Limit:").pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Label(TopFrame, text=" ").pack(side = LEFT)

Label(TopFrame, text="Day Limit:").pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Label(TopFrame, text=" ").pack(side = LEFT)

Label(TopFrame, text="%Change:").pack(side = LEFT)
R1 = Radiobutton(TopFrame, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(TopFrame, text="*50%", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(TopFrame, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(TopFrame, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

RunPrediction = Button(TopFrame, text ="Execute", state="normal", command = startUpdatePredictor)
RunPrediction.pack(side = LEFT)

Reset = Button(TopFrame, text ="Stop", state="disable", command = Resetfun)
Reset.pack(side = LEFT)

PathSelect = Button(TopFrame, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)
SelectedPath = Label(TopFrame, text=Path)
SelectedPath.pack(side = LEFT)

#P = ttk.Progressbar(TopFrame, orient="horizontal", length=300, maximum=100, mode="determinate")
#P.pack()


Label(root).pack()
root.title(str(Version+" "+Title +" by "+Author))
root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()



In [ ]:
#Testout function ending trigger
from tkinter import *
import time
import threading

def Close():
    global GlobalStatus
    GlobalStatus = "disable"
    status.config(text=GlobalStatus)
    root.destroy()

def Reset():
    global GlobalStatus
    GlobalStatus = "disable"
    status.config(text=GlobalStatus)
    
def UpdatePredictor():
    global GlobalStatus
    global count
    GlobalStatus = "normal"
    for i in range(1,100):
        if GlobalStatus == "normal":
            count = i
            print(count)
            time.sleep(1)
        else:
            break

def startUpdatePredictor():
    t1 = threading.Thread(target=UpdatePredictor,daemon=True).start()


root = Tk()
# root is your root window

GlobalStatus = "normal"
count = 0

Runer = Button(root, text ="Run", command = startUpdatePredictor)
Runer.pack(side = LEFT)

Stopper = Button(root, text ="Stop", command = Reset)
Stopper.pack(side = LEFT)

status = Label(root, text=GlobalStatus)
status.pack(side = LEFT)

root.protocol('WM_DELETE_WINDOW', Close)
root.mainloop()

In [ ]:
#Enable output file path specification
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter.messagebox import showinfo
import time

#Program Data
Title= "Update Schedule Predictor"
Author= "Abul Hassan Sheikh"
Version= "V1.0"
Date= "October 2019"


def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    selection = (year, wLimit, dThresh, perChange, Path)
    print(selection)
    process = 0
    
    #for p in range(1,100):
    #    time.sleep(2.4)
    #    pPer = p
    #    process += pPer
    #update()

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)
    
def update(): #Reference: https://stackoverflow.com/questions/55760066/stop-a-progress-bar-at-max-value-tkinter
    global process
    P['value'] = process
    if P['value'] >= P['maximum']:
        popup_showinfo()
        return  #EndForLoop
    root.after( 100, update )

    
def popup_showinfo():
    showinfo(Title, "Update Prediction Complete")

#Make Main window    
root = Tk()

#Set Variables
get_perChange = IntVar()
Path = 'C:/'

#Make the frames
TopFrame = Frame(root)
TopFrame.pack()

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

left = Label(TopFrame, text="Parameters:")
left.pack(side = LEFT)

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

h = Label(TopFrame, text="Year:")
h.pack(side = LEFT)
get_year.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

m = Label(TopFrame, text="Weekly Limit:")
m.pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

s = Label(TopFrame, text="Day Limit:")
s.pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

left = Label(TopFrame, text="%Change:")
left.pack(side = LEFT)

R1 = Radiobutton(TopFrame, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(TopFrame, text="*50%", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(TopFrame, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(TopFrame, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

RunPrediction = Button(TopFrame, text ="Run Prediction", command = UpdatePredictor)
RunPrediction.pack(side = LEFT)

PathSelect = Button(TopFrame, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)

SelectedPath = Label(TopFrame, text=Path)
SelectedPath.pack(side = LEFT)
#P = ttk.Progressbar(TopFrame, orient="horizontal", length=300, maximum=100, mode="determinate")
#P.pack()


label = Label(root)
label.pack()


root.title(str(Version+" "+Title +" by "+Author))
root.mainloop()



In [ ]:
#Print change in path
from tkinter import filedialog
from tkinter import *

Path = 'C:/'

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()
    SelectedPath.config(text = Path)

def UpdatePredictor():
    print(Path)

root = Tk()

PathSelect = Button(root, text ="Select Path", command = FolderSelection)
PathSelect.pack(side = LEFT)

BFTime = Button(root, text ="PrintPath", command = UpdatePredictor)
BFTime.pack(side = LEFT)

SelectedPath = Label(root, text=Path)
SelectedPath.pack(side = LEFT)

root.mainloop()

In [ ]:
#Create folder selection and use dialog
from tkinter import filedialog
from tkinter import *

Path = 'C:/'

def FolderSelection():
    global Path
    Path = filedialog.askdirectory()

def UpdatePredictor():
    print(Path)

root = Tk()

BFTime = Button(root, text ="Select Path", command = FolderSelection)
BFTime.pack(side = LEFT)

BFTime = Button(root, text ="PrintPath", command = UpdatePredictor)
BFTime.pack(side = LEFT)

#root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
#print (root.filename)
root.mainloop()

In [ ]:
from tkinter import filedialog
from tkinter import *
root = Tk()
Path = filedialog.askdirectory()
print (Path)
root.mainloop()

In [ ]:
#BUtton works and input can be sent to target function
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
import time


def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    selection = (year, wLimit, dThresh, perChange)
    print(selection)
    process = 0
    
    #for p in range(1,100):
    #    time.sleep(2.4)
    #    pPer = p
    #    process += pPer
    #update()

    
def update(): #Reference: https://stackoverflow.com/questions/55760066/stop-a-progress-bar-at-max-value-tkinter
    global process
    P['value'] = process
    if P['value'] >= P['maximum']:
        popup_showinfo()
        return  #EndForLoop
    root.after( 100, update )

    
def popup_showinfo():
    showinfo("Window", "Update Prediction Complete")

    
root = Tk()
get_perChange = IntVar()


#Make the frames
TopFrame = Frame(root)
TopFrame.pack()

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

left = Label(TopFrame, text="Parameters:")
left.pack(side = LEFT)

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

h = Label(TopFrame, text="Year:")
h.pack(side = LEFT)
get_year.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

m = Label(TopFrame, text="Weekly Limit:")
m.pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

s = Label(TopFrame, text="Day Limit:")
s.pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

left = Label(TopFrame, text="%Change:")
left.pack(side = LEFT)

R1 = Radiobutton(TopFrame, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(TopFrame, text="*50%", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(TopFrame, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(TopFrame, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

BFTime = Button(TopFrame, text ="Run Prediction", command = UpdatePredictor)
BFTime.pack(side = LEFT)


#P = ttk.Progressbar(TopFrame, orient="horizontal", length=300, maximum=100, mode="determinate")
#P.pack()


label = Label(root)
label.pack()

root.title('Update Schedule Predictor')
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter.messagebox import showinfo

def popup_bonus():
    win = tk.Toplevel()
    win.wm_title("Window")

    l = tk.Label(win, text="Input")
    l.grid(row=0, column=0)

    b = ttk.Button(win, text="Okay", command=win.destroy)
    b.grid(row=1, column=0)

def popup_showinfo():
    showinfo("Window", "Hello World!")

class Application(ttk.Frame):

    def __init__(self, master):
        ttk.Frame.__init__(self, master)
        self.pack()

        self.button_bonus = ttk.Button(self, text="Bonuses", command=popup_bonus)
        self.button_bonus.pack()

        self.button_showinfo = ttk.Button(self, text="Show Info", command=popup_showinfo)
        self.button_showinfo.pack()

root = tk.Tk()

app = Application(root)

root.mainloop()

In [ ]:
#figure out the layout of the display
from tkinter import *
from tkinter import ttk


def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    selection = (year, wLimit, dThresh, perChange)
    
    process = 0
    
    for p in range(1,100):
        print(p)
        process += p
    update()

    
def update(): #Reference: https://stackoverflow.com/questions/55760066/stop-a-progress-bar-at-max-value-tkinter
    global process
    P['value'] = process
    if P['value'] >= P['maximum']:

        # Here you can do anything when the  progress bar finishes.

        return  # This will end the after() loop
    root.after( 100, update )
    
root = Tk()
get_perChange = IntVar()



#Make the frames
TopFrame = Frame(root)
TopFrame.pack()

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

left = Label(TopFrame, text="Parameters:")
left.pack(side = LEFT)

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

h = Label(TopFrame, text="Year:")
h.pack(side = LEFT)
get_year.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

m = Label(TopFrame, text="Weekly Limit:")
m.pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

s = Label(TopFrame, text="Day Limit:")
s.pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

left = Label(TopFrame, text="%Change:")
left.pack(side = LEFT)

R1 = Radiobutton(TopFrame, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(TopFrame, text="*50%", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(TopFrame, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(TopFrame, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

BFTime = Button(TopFrame, text ="Run Prediction", command = UpdatePredictor)
BFTime.pack(side = LEFT)


P = ttk.Progressbar(TopFrame, orient="horizontal", length=300, maximum=100, mode="determinate")
P.pack()


label = Label(root)
label.pack()

root.title('Update Schedule Predictor')
root.mainloop()


In [ ]:
from tkinter import ttk
from tkinter import *

def update():
    global process
    process += 10
    P['value'] = process
    if P['value'] >= P['maximum']:

        # Here you can do anything when the  progress bar finishes.

        return  # This will end the after() loop
    root.after( 100, update )

root = Tk()

P = ttk.Progressbar(root, orient="horizontal", length=300, maximum=100, mode="determinate")
P.pack()

L = Label(root, text="In Process")
L.pack()

process = 0


update()

root.mainloop()

In [ ]:
#figure out the layout of the display
from tkinter import *


def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    perChange = int(get_perChange.get())
    selection = (year, wLimit, dThresh, perChange)
    for p in range(1,100):
        label.config(text = p)
    
    
root = Tk()
get_perChange = IntVar()

#Make the frames
TopFrame = Frame(root)
TopFrame.pack()

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

left = Label(TopFrame, text="Parameters:")
left.pack(side = LEFT)

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

h = Label(TopFrame, text="Year:")
h.pack(side = LEFT)
get_year.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

m = Label(TopFrame, text="Weekly Limit:")
m.pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

s = Label(TopFrame, text="Day Limit:")
s.pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

left = Label(TopFrame, text="%Change:")
left.pack(side = LEFT)

R1 = Radiobutton(TopFrame, text="25%", variable=get_perChange, value=25)
R1.pack(side = LEFT)

R2 = Radiobutton(TopFrame, text="*50%", variable=get_perChange, value=50)
R2.pack(side = LEFT)

R3 = Radiobutton(TopFrame, text="75%", variable=get_perChange, value=75)
R3.pack(side = LEFT)

R4 = Radiobutton(TopFrame, text="100%", variable=get_perChange, value=100)
R4.pack(side = LEFT)

BFTime = tkinter.Button(TopFrame, text ="Run Prediction", command = UpdatePredictor)
BFTime.pack(side = LEFT)


label = Label(root)
label.pack()

root.title('Update Schedule Predictor')
root.mainloop()


In [ ]:
def sel():
   selection = "Value = " + str(var.get())
   label.config(text = selection)

root = Tk()
var = DoubleVar()

left = Label(root, text="%Change:")
left.pack(side = LEFT)

scale = Scale(root, orient=HORIZONTAL, variable = var, from_ = 0, to = 100 )
scale.pack(anchor=CENTER)

#button = Button(root, text="Get Scale Value", command=sel)
#button.pack(anchor=CENTER)

label = Label(root)
label.pack()

root.mainloop()

In [ ]:
def sel():
   selection = "You selected the option " + str(var.get())
   label.config(text = selection)

root = Tk()
var = IntVar()

R1 = Radiobutton(root, text="Option 1", variable=var, value=1,
                  command=sel)
R1.pack( anchor = W )

R2 = Radiobutton(root, text="Option 2", variable=var, value=2,
                  command=sel)
R2.pack( anchor = W )

R3 = Radiobutton(root, text="Option 3", variable=var, value=3,
                  command=sel)
R3.pack( anchor = W)

label = Label(root)
label.pack()
root.mainloop()

In [ ]:
#figure out the layout of the display
from tkinter import *


def UpdatePredictor():
    year = int(get_year.get())
    wLimit = int(get_wLimit.get())
    dThresh = int(get_dThresh.get())
    print(year, wLimit, dThresh)
    
    
root = Tk()
var = IntVar()

#Make the frames
TopFrame = Frame(root)
TopFrame.pack()

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

year = int(get_year.get())
wLimit = int(get_wLimit.get())
dThresh = int(get_dThresh.get())

left = Label(TopFrame, text="Parameters:")
left.pack(side = LEFT)

get_year = Spinbox(TopFrame, from_=2000, to=2100, width = 4)
get_wLimit = Spinbox(TopFrame, from_=1, to=100, width = 2)
get_dThresh = Spinbox(TopFrame, from_=1, to=99, width = 2)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

h = Label(TopFrame, text="Year:")
h.pack(side = LEFT)
get_year.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

m = Label(TopFrame, text="Weekly Limit:")
m.pack(side = LEFT)
get_wLimit.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

s = Label(TopFrame, text="Day Limit:")
s.pack(side = LEFT)
get_dThresh.pack(side = LEFT)

Sp = Label(TopFrame, text=" ")
Sp.pack(side = LEFT)

left = Label(TopFrame, text="%Change:")
left.pack(side = LEFT)

R1 = Radiobutton(TopFrame, text="25%", variable=var, value=1)
R1.pack(side = LEFT)

R2 = Radiobutton(TopFrame, text="*50%", variable=var, value=2)
R2.pack(side = LEFT)

R3 = Radiobutton(TopFrame, text="75%", variable=var, value=2)
R3.pack(side = LEFT)

R4 = Radiobutton(TopFrame, text="100%", variable=var, value=2)
R4.pack(side = LEFT)

BFTime = tkinter.Button(TopFrame, text ="Run Prediction", command = UpdatePredictor)
BFTime.pack(side = LEFT)


label = Label(root)
root.title('Update Schedule Predictor')
root.mainloop()


## Algorithm

In [ ]:
pd.options.mode.chained_assignment = None
import statistics as st
import calendar

DueDatePrediction = pd.DataFrame(columns = ['attribute_set',"MNum",'Month' , 'Day', 
                                                "Jan", "Feb", "Mar", "Apr", "May", "June", "July", "Aug", 
                                                "Sep", "Oct", "Nov", "Dec","TotalOAD","test1"])

def trend(value):
    
for value in range(len(AllBrands)-1):

    brand = AllBrands[value] 
    BrandYearSS = monthlyGroupingAll[(monthlyGroupingN['attribute_set']==brand)].reset_index(drop=True)

    BrandYearSS["OverSales"] =  BrandYearSS["count"] - st.mean(BrandYearSS["count"])
    BrandYearSS["OverRev"] = BrandYearSS["NetRevenue"] -  st.mean(BrandYearSS["NetRevenue"]) 

    OverIdentify = BrandYearSS[(BrandYearSS["OverSales"]>0) & (BrandYearSS["OverRev"]>0)]
    OverIdentify_Tally = OverIdentify.groupby(["OD_MonthNum"], as_index=False)['OD_MonthNum'].agg({"OverCounter":"count"})
    
    OverIdentify_Final = pd.DataFrame({"OD_MonthNum":["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]})
    OverIdentify_Final = OverIdentify_Final.merge(OverIdentify_Tally, on=('OD_MonthNum'), how = "outer").reset_index(drop=True)
    OverIdentify_Final = OverIdentify_Final.fillna(0)
             
    if len(OverIdentify_Final)>1:
    
        for m in range(len(OverIdentify_Final)-1):
            Base = (OverIdentify_Final.loc[m+1, "OverCounter"])
            Threshold = (OverIdentify_Final.loc[m, "OverCounter"]*1.5)
            if Base>Threshold:
                if Base>=6:###Consider reducing to 6
                    OverIdentify_Final.loc[m, "test1"] = "Update"
                else:
                    OverIdentify_Final.loc[m, "test1"]  = ""
            else:
                OverIdentify_Final.loc[m, "test1"]  = ""

        FindDup = len(OverIdentify_Final[OverIdentify_Final["test1"]=="Update"])
        
        if FindDup > 0:
            if FindDup==1:
                OverIdentify_Final["test2"] = OverIdentify_Final["test1"]

                WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)

            elif FindDup>1:
                MultiUpdate = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"]
                MaxMonths = (len(OverIdentify_Final)-1 - MultiUpdate.index[-1])+1
                MultiUpdate["index"] = MultiUpdate.index
                MultiUpdate = MultiUpdate.reset_index(drop=True)

                for s in range(len(MultiUpdate)):
                    if s != (len(MultiUpdate)-1):
                        IndexVal = MultiUpdate.loc[s,"index"]
                        MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal), "OverCounter"])
                    else: 
                        IndexVal = MultiUpdate.loc[s,"index"]
                        MultiUpdate.loc[s, "Value"] = sum(OverIdentify_Final.loc[IndexVal+1:(MaxMonths+IndexVal-1), "OverCounter"], OverIdentify_Final.loc[0, "OverCounter"])                  

                WinMonth = MultiUpdate.sort_values("Value", ascending = False).reset_index(drop=True).loc[0,"index"]
                
                for m in range(len(OverIdentify_Final)-1):
                    if m == WinMonth:
                        OverIdentify_Final.loc[WinMonth, "test2"] = "Update"
                    else:
                        OverIdentify_Final.loc[m, "test2"]  = ""

                WinningMonth = OverIdentify_Final[OverIdentify_Final.test2 == "Update"].reset_index(drop=True).loc[0,"OD_MonthNum"]
                SubsetWinMonth = monthlyGroupingAll[(monthlyGroupingAll.attribute_set== brand) & (monthlyGroupingAll.OD_MonthNum== WinningMonth)].reset_index(drop=True)

        #Determine day of WinningMonth of due date
            TextMonth = calendar.month_name[int(WinningMonth)]
            MaxCountIndex = SubsetWinMonth[SubsetWinMonth["count"] == (SubsetWinMonth["count"].max())].index[0]
            MaxRevIndex = SubsetWinMonth[SubsetWinMonth["NetRevenue"] == (SubsetWinMonth["NetRevenue"].max())].index[0]

            if MaxCountIndex == MaxRevIndex:
                MaxOfMonth = MaxCountIndex
                if MaxOfMonth == len(SubsetWinMonth)-1:
                    MinusLast = SubsetWinMonth.loc[0:len(SubsetWinMonth)-2]
                    MaxCountIndex = MinusLast[MinusLast["count"] == (MinusLast["count"].max())].index[0]
                    MaxRevIndex = MinusLast[MinusLast["NetRevenue"] == (MinusLast["NetRevenue"].max())].index[0]
                    if MaxCountIndex == MaxRevIndex:
                        MaxOfMonth = MaxCountIndex
                        MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                    else: 
                        MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                        MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
                else:
                    MinSubset = SubsetWinMonth.loc[MaxOfMonth:]
            else: 
                MaxOfMonth = min(MaxCountIndex, MaxRevIndex)
                MinSubset = SubsetWinMonth.loc[MaxOfMonth:]

            MinCountIndex = MinSubset[MinSubset["count"]==MinSubset.loc[MaxOfMonth:, "count"].min()].index[-1]
            MinRevIndex =   MinSubset[MinSubset["NetRevenue"]==MinSubset.loc[MaxOfMonth:, "NetRevenue"].min()].index[-1]    

            if MinCountIndex == MinRevIndex:
                FinalIndex = MinRevIndex
                SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
            else: 
                FinalIndex = min(MinCountIndex, MinRevIndex)
                SubsetWinMonth.loc[FinalIndex,"DeadLine"] = "***"
                WinDay = SubsetWinMonth.loc[FinalIndex, "OD_MonthDay"]
        else:
            TextMonth = "None"
            WinningMonth = 0
            WinDay = 0
        
    #Formatting Data
    AllRecomMon = str()
    AllRecomMonRou = OverIdentify_Final[OverIdentify_Final["test1"]=="Update"].reset_index(drop = True)
    for m in range(len(AllRecomMonRou)):
        AllRecomMon = AllRecomMon + calendar.month_name[int(AllRecomMonRou.loc[m,"OD_MonthNum"])]+ "-"
            
    OC = OverIdentify_Final["OverCounter"]
    TotalOAD = sum(OC)
    DueDateData = brand,WinningMonth,TextMonth,WinDay,OC[0],OC[1],OC[2],OC[3],OC[4],OC[5],OC[6],OC[7],OC[8],OC[9],OC[10],OC[11],TotalOAD,str(AllRecomMon)
    
    DueDatePrediction = DueDatePrediction.append(pd.Series(DueDateData, index=DueDatePrediction.columns),  ignore_index=True)
            
            #print("------------------------")
    print(brand + " " + str(FindDup))
            #print(OverIdentify_Final)
            #SubsetWinMonth = SubsetWinMonth.fillna("")
            #print(DueDatePrediction)
            
            
            

In [ ]:
for i in range(len(AllBrands)-1):
    trend(i, DueDatePrediction)

DueDatePrediction.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\DueDatePrediction.csv', index = None, header=True) 
#Don't forget to add '.csv' at the end of the path

In [ ]:
OverIdentify_Final
WinningMonth
value = 16


count = 0
sumDays = 0
dayCount = 0

for i in range(int(WinningMonth), len(OverIdentify_Final)):
    if i != 12:
        month = OverIdentify_Final.loc[i, "OD_MonthNum"]
        days = OverIdentify_Final.loc[i, "OverCounter"]
        count = count + 1
        sumDays = sumDays + days
        avg = (sumDays/count)*.75

        if(days >= avg):
            dayCount = dayCount + 30
        else:
            break
    elif i == 12:
        month = OverIdentify_Final.loc[0, "OD_MonthNum"]
        days = OverIdentify_Final.loc[0, "OverCounter"]
        count = count + 1
        sumDays = sumDays + days
        avg = (sumDays/count)*.75

        if(days >= avg):
            dayCount = dayCount + 30
        else:
            break
        
print(dayCount)
        
        
        



In [ ]:
trend(209)
value = "WeatherTech"
value=336
brand = AllBrands[26] 
brand = "StopTech"

AllBrands.index(1)
np.where(AllBrands=="TowReady")

In [ ]:
pd.options.mode.chained_assignment = None

brand = AllBrands[2] 
BrandYearSS = monthlyGroupingAll[(monthlyGroupingN['attribute_set']==brand)].reset_index(drop=True)
AllYears = BrandYearSS.OD_Year.unique()
year = AllYears[1]

pooledTrend = pd.DataFrame(columns = ['year' , 'month', 'type' , 'rank', "value"])

for y in AllYears:

    SingleYear = BrandYearSS[BrandYearSS['OD_Year']==y]
    if SingleYear.count()[0] == 12:
        
        #Sales Calc
        totalS = SingleYear['count'].sum()
        SingleYear['PerSales'] = (SingleYear['count']/totalS)*100
        tms = list((SingleYear.sort_values(["PerSales","OD_MonthNum"] , 
                                      ascending = [False, True]).reset_index(drop=True).iloc[0, 2:4]))
        sms = (SingleYear.sort_values(["PerSales","OD_MonthNum"] , 
                                      ascending = [False, True]).reset_index(drop=True).iloc[1, 2:4])
        TopmonthS = y, tms[0], "Sales", "Top", tms[1]
        SecmonthS = y, sms[0], "Sales", "Sec", sms[1] 
       
        #RevCalc
        totalR = SingleYear['NetRevenue'].sum()
        SingleYear['PerRev'] = (SingleYear['NetRevenue']/totalR)*100
        tmr = (SingleYear.sort_values(["PerRev","OD_MonthNum"] , 
                                      ascending = [False, True]).reset_index(drop=True).iloc[0, 2:5])
        smr = (SingleYear.sort_values(["PerRev","OD_MonthNum"] , 
                                      ascending = [False, True]).reset_index(drop=True).iloc[1, 2:5])   
        TopmonthR = y,tmr[0], "Rev", "Top", tmr[2]
        SecmonthR = y,smr[0], "Rev", "Sec", smr[2]
        
        allresults = [pd.Series(TopmonthS, index=pooledTrend.columns ),  
                      pd.Series(TopmonthR, index=pooledTrend.columns ),
                      pd.Series(SecmonthS, index=pooledTrend.columns ), 
                      pd.Series(SecmonthR, index=pooledTrend.columns )]

        #allresults = [TopmonthS, TopmonthR, SecmonthR, SecmonthS]
        pooledTrend = pooledTrend.append(allresults,  ignore_index=True)

        
Rev = pooledTrend[pooledTrend["type"]=="Rev"]
Sales = pooledTrend[pooledTrend["type"]=="Sales"]


(Rev.groupby(["month"], as_index=False)
                ['value'].
                agg({"RevSum":"sum"}).sort_values(["month","RevSum"] , ascending = [True, True]))

(Sales.groupby(["month"], as_index=False)
                ['value'].
                agg({"SalesSum":"sum"}).sort_values(["month","SalesSum"] , ascending = [True, True]))

pooledTrend.month.value_counts().sort_values()



BrandYearSS['count'].iloc[0]

pd.Series(BrandYearSS['count']).pct_change()
print [100 * (b - a) / a for a, b in zip(BrandYearSS['CumSum'][::1], BrandYearSS['CumSum'][1::1])]

BrandFilterSD['PerSales'] =BrandFilterSD.apply(lambda BrandFilterSD: BrandFilterSD.count/total, axis = 1) 

In [ ]:
#Predict when to perform pricing update depending on day of month using logistic regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


Path = "//192.168.2.32/Group/Data Team/Brand_Update_Location/5_R_Brand_Reference_Files/Brand Launch Calendar - Event_Tracker.csv"
BLC = pd.read_csv(Path, encoding='mac_roman')
BLC = BLC[["Date", "Brand", "Category", "Notes"]]

BLC['Date'] = pd.to_datetime(BLC['Date'], format="%d-%m-%y")
BLC['Day'] = pd.DatetimeIndex(BLC['Date']).day



# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(BLC[['Height','Weight']].values, 
                                              (BLC.Category == "Male").values,random_state=5)

clf = LogisticRegression(solver='lbfgs')
# Fit the model on the trainng data.
clf.fit(Xlr, ylr)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtestlr), ytestlr))


# TroubleShooting